In [11]:
import requests
import pandas as pd
import os
from config import *
import json

In [2]:
with requests.Session() as session:
    general_response = session.get(general_endpoint).json()
    league_response = session.get(league_endpoint).json()
    fixture_response = session.get(fixture_endpoint).json()